# Bees vs Wasps notebook

#### Imports

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# additional classic imports
from pathlib import Path
import pandas as pd
import numpy as np
import random

#### Run parameters

In [4]:
bs = 64 # Batch size
resize_size = 96 # for training, resize all the images to a square of this size
training_subsample = 0.1 # for development, use a small fraction of the entire dataset rater than full dataset

In [5]:
bees_vs_wasps_dataset_path=Path('./data') # this is relative to the "example_notebook" folder. Modify this to reflect your setup
df_labels = pd.read_csv(bees_vs_wasps_dataset_path/'labels.csv')
df_labels=df_labels.set_index('id')
# perform dataset subsampling
df_labels = df_labels.sample(frac=training_subsample, axis=0)

In [6]:
df_labels

,path,is_bee,is_wasp,is_otherinsect,is_other,photo_quality,is_validation,is_final_validation,label
id,,,,,,,,,
11025,other_noinsect\544122.jpg,0,0,0,1,0,0,0,other
5827,wasp2\F00331.jpg,0,1,0,0,0,0,1,wasp
10209,other_insect\6168096467_6d62862eef_n.jpg,0,0,1,0,1,1,0,insect
1674,bee1\5139320928_a35973bcb3_m.jpg,1,0,0,0,1,0,0,bee
6445,wasp2\H00098.jpg,0,1,0,0,0,0,0,wasp
...,...,...,...,...,...,...,...,...,...
2906,bee2\P18775-17r.jpg,1,0,0,0,0,0,1,bee
10537,other_insect\9479634782_e63e0b3543_n.jpg,0,0,1,0,1,0,1,insect
402,bee1\18750162506_8733f5bb51_n.jpg,1,0,0,0,1,0,0,bee
